<a href="https://colab.research.google.com/github/darinaksena/hpc/blob/main/ga_test_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import matplotlib.pyplot as plt
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numba
import copy
import numpy as np
import time


solution = np.array([3, 2.5, 1, 4.5, 0.5]) 

deg_p = 5

count = 1000
population_size = (count, deg_p) 
# new_population = np.zeros(population_size)

generations_count = 5000
parents_count = 50

X = np.linspace(0,np.pi, 20)

In [2]:
def fitness(individuals, points):
    fitness = []
    poly2 = np.poly1d(solution)
    for individual in individuals:
      s = 0.
      poly = np.poly1d(individual)
      s += np.linalg.norm(individual-solution)
      # for point in points:
        # s = np.power(poly(point) - poly2(point),2)
      fitness.append(s)
    return fitness

def selection(individuals, fit, num_parents):
    parents = np.empty((num_parents, individuals.shape[1]))
    for parent_num in range(num_parents):
        i = np.argmin(fit)
        parents[parent_num, :] = individuals[i, :]
        fit[i] = np.inf
    return parents

def crossover(parents):
    individuals = np.empty(population_size)
    individuals[:parents.shape[0], :] = parents
    crosspoint = int(np.random.choice(population_size[1],1))

    for k in range(parents.shape[0], population_size[0], 2):
        p1 = np.random.choice(parents.shape[0], 1)
        p2 = np.random.choice(parents.shape[0], 1)
        individuals[k, :crosspoint] = parents[p1, :crosspoint]
        individuals[k, crosspoint:] = parents[p2, crosspoint:]

        individuals[k+1, :crosspoint] = parents[p2, :crosspoint]
        individuals[k+1, crosspoint:] = parents[p1, crosspoint:]
    return individuals

def mutation(individuals): 
  # first individual is usually left without changes to keep 
  # the best individual mutNumber = random between second_individual_start_bit and ←- 
  mutNumber = np.random.randint( 1, individuals.shape[0] )# usually from 1% to 2% of bits in individuals
  mutIdx =  np.random.randint( 1, individuals.shape[1] ,mutNumber)
  for i in mutIdx:# times { 
    number_of_bit_to_mutate = np.random.randint(0,individuals.shape[1]) 
    individuals[i,number_of_bit_to_mutate] = np.random.uniform(-1.0, 1.0, 1)
  return individuals

In [ ]:
individuals = np.zeros(population_size)
for generation in range(generations_count):
    fitness_array = fitness(individuals, X)
    # print(fitness)
    parents = selection(individuals, fitness_array, parents_count)
    
    individuals = crossover(parents)
    
    individuals = mutation(individuals) 

In [4]:
fit = fitness(individuals, X)
print('Fitness:', np.round(np.min(fit),2))
idx = np.argmin(fit)
best_individuals = individuals[idx]

print("Best individuals:\t", np.round(best_individuals,1))
print("Solution:\t\t", solution)

Fitness: 4.31
Best individuals:	 [1.  1.  1.  1.  0.4]
Solution:		 [3.  2.5 1.  4.5 0.5]


In [5]:
@cuda.jit
def fitness_gpu(true_weights, individuals, fitness_array, points):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    i = tx + bx * bw

    # poly2 = np.poly1d(true_weights)
    for i in range(len(individuals)):
      s = 0.
      # poly = np.poly1d(individual)
      for j in range (len(true_weights)):
        # s += cuda.linalg.norm(individuals[i,j]-true_weights[j])
        s += (individuals[i,j]-true_weights[j]) ** 2
      # for point in points:
        # s = np.power(poly(point) - poly2(point),2)
        numba.cuda.syncthreads()
      fitness_array[i] = s
      numba.cuda.syncthreads()
      # fitness_array.append(s)

In [10]:
@numba.njit(parallel=True, fastmath=True)
def selection_gpu(pop, fitness, parents_count):
    parents = np.empty((parents_count, pop.shape[1]))
    for parent_num in numba.prange(parents_count):
        max_fitness_idx = np.argmin(fitness)
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = np.inf
    return parents

In [7]:
@cuda.jit
def crossover_gpu(parents, offspring):
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    i = tx + bx * bw

    crossover_point = np.uint8(offspring.shape[1]/2)
    inverse_point = np.uint8(offspring.shape[1] - crossover_point)

    if i < offspring.shape[0]:
        parent1_idx = i % parents.shape[0]
        parent2_idx = (i + 1) % parents.shape[0]

        for point in range(crossover_point):
            offspring[i, point] = parents[parent1_idx, point]
            offspring[i, -point] = parents[parent2_idx, -point]
            numba.cuda.syncthreads()
            
        for point in range(inverse_point):
            offspring[i, offspring.shape[1]-point-1] = parents[parent1_idx, offspring.shape[1]-point-1]
            numba.cuda.syncthreads()

    # for i in range(parents.shape[0]):
    #   for j in range(parents.shape[1]):
    #     offspring[i,j] = parents[i,j]
    #     numba.cuda.syncthreads()
    # # crossover_point = np.random.randint()#(0,population_size[1])
    # crossover_point = population_size[0]/2

    # for k in range(parents.shape[0], population_size[0], 2):
    #     # parent1_idx = np.random.randint(0,parents.shape[0])
    #     # parent2_idx = np.random.randint(0,parents.shape[0])
    #     for j in range(crossover_point):
    #       offspring[k, j] = parents[k, j]
    #       offspring[k+1, j] = parents[k+1, j]
    #       offspring[k, -j] = parents[k+1, -j]
    #       offspring[k+1, -j] = parents[k, -j]
    #       numba.cuda.syncthreads()
    #     numba.cuda.syncthreads()

In [8]:
def mutation(individuals): 
  # first individual is usually left without changes to keep 
  # the best individual mutNumber = random between second_individual_start_bit and ←- 
  mutNumber = np.random.randint( 1, individuals.shape[0] )# usually from 1% to 2% of bits in individuals
  mutIdx =  np.random.randint( 1, individuals.shape[1] ,mutNumber)
  for i in mutIdx:# times { 
    number_of_bit_to_mutate = np.random.randint(0,individuals.shape[1]) 
    individuals[i,number_of_bit_to_mutate] = np.random.uniform(-1.0, 1.0, 1)
  return individuals

In [ ]:
individuals = np.zeros(population_size)
threads_per_block = 256
blocks_per_grid = (count * deg_p + (threads_per_block - 1))

fit = np.zeros((count, ))
new_individuals = np.zeros(population_size).astype(np.float32)
rng_states = create_xoroshiro128p_states(threads_per_block * blocks_per_grid, seed=1)

for generation in range(generations_count):
    fitness_gpu[blocks_per_grid, threads_per_block](solution, individuals, fit, X)

    parents = selection_gpu(individuals, fit, 
                                      parents_count).astype(np.float32)
    
    crossover_gpu[blocks_per_grid, threads_per_block](parents,
                                       new_individuals)
  
    individuals = mutation(new_individuals)

In [13]:
fit = fitness(individuals,X)
print('Fitness:', np.round(np.min(fit),2))
idx = np.argmin(fit)
best_individuals = individuals[idx]

print("Best individuals:\t", np.round(best_individuals,1))
print("Solution:\t\t", solution)


Fitness: 4.57
Best individuals:	 [1.  0.8 0.2 0.9 0.7]
Solution:		 [3.  2.5 1.  4.5 0.5]
